# StyleGAN identity perturbations

This code is based on the StyleGAN `pretrained_example.py` script

In [1]:
import os
import pickle
import numpy as np
import PIL.Image
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import dnnlib
import dnnlib.tflib as tflib
import config
import imageio
import cv2

In [2]:
import random
import string
def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))

In [3]:
# create directory for results, if needed
os.makedirs('results', exist_ok=True)

In [4]:
def setup():
    # Initialize TensorFlow.
    tflib.init_tf()
    # Load pre-trained network.
    url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
    with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
        _G, _D, Gs = pickle.load(f)
    return [_G, _D, Gs]

In [5]:
[_G, _D, Gs] = setup()

In [6]:
# create a nice random generator state. Replace None with a constant int to
# reproduce the same result
rnd = np.random.RandomState(None)

In [12]:
# Exp 1:
# perturb EACH of the coefficients in the latent vector by a different random amount
# a standard normal multiplied by a shared gain factor.
# generated images are written to a video file
# diversity of images depends on the value of "magnitude".  0.1 tends to preserve
# quasi-identity, gender, expression, pose. Larger values generate variety in all
# attributes.
# You can run this cell multiple times and get a different output video.

# number of images to make
num_samples = 40

latent = rnd.randn(Gs.input_shape[1])

stk = []
magnitude = 0.1 # modulate all normal perturbations by this amount

for i in range(num_samples):
    l2 = latent + magnitude * rnd.randn(Gs.input_shape[1])   # perturb
    stk.append(l2)
latents = np.stack(stk)
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
# Render
images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)
s=images[0].shape

# write output images to a video file
vfn =  os.path.join('results', 'exp1_'+randomString()+'.avi')
videowriter =  cv2.VideoWriter(vfn,cv2.VideoWriter_fourcc(*'mp4v'), 10, (s[1],s[0]))
for i in range(num_samples):
    videowriter.write(images[i][...,::-1])
videowriter.release()

num_items: 40


In [22]:
# EXP2:
# perturb just one element of the latent vector
rnd = np.random.RandomState(None)

num_samples = 55

latent = rnd.randn(Gs.input_shape[1])

stk = []
magnitude = 10

index = 399

for i in range(num_samples):
    l2 = latent
    l2[index] = l2[index] + magnitude * rnd.randn()
    stk.append(l2)
latents = np.stack(stk)
#latents = np.stack([rnd.randn(Gs.input_shape[1]) for _ in range(num_samples)])
#latents = rnd.randn(1, Gs.input_shape[1])
#latents = np.stack([rnd.randn(Gs.input_shape[1])] * 2)
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

# Render
images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)
s=images[0].shape

vfn =  os.path.join('results', 'exp2_'+randomString()+'_{0}_{1}.avi'.format(index,magnitude))
videowriter =  cv2.VideoWriter(vfn,cv2.VideoWriter_fourcc(*'mp4v'), 10, (s[1],s[0]))
for i in range(num_samples):
    videowriter.write(images[i][...,::-1])
videowriter.release()
#    PIL.Image.fromarray(images[i], 'RGB').save(png_filename)

num_items: 55


In [9]:
# Exp 1b:
# perturb EACH of the coefficients in the latent vector by a different random amount
# a standard normal multiplied by a shared gain factor.

# turn off randomize_noise

# generated images are written to a video file
# diversity of images depends on the value of "magnitude".  0.1 tends to preserve
# quasi-identity, gender, expression, pose. Larger values generate variety in all
# attributes.
# You can run this cell multiple times and get a different output video.

# number of images to make
num_samples = 40

latent = rnd.randn(Gs.input_shape[1])

stk = []
magnitude = 5000 # modulate all normal perturbations by this amount

for i in range(num_samples):
    l2 = latent + magnitude * rnd.randn(Gs.input_shape[1])   # perturb
    stk.append(l2)
latents = np.stack(stk)
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
# Render
images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)
s=images[0].shape

# write output images to a video file
vfn =  os.path.join('results', 'exp1b_'+randomString()+'.avi')
videowriter =  cv2.VideoWriter(vfn,cv2.VideoWriter_fourcc(*'mp4v'), 10, (s[1],s[0]))
for i in range(num_samples):
    videowriter.write(images[i][...,::-1])
videowriter.release()

num_items: 40


In [ ]:
# idea: discover latent vectors for everyone in a data set. Where do they live?